In [2]:
import nltk
import pandas as pd
from bs4 import BeautifulSoup
import json
import lxml
import pandas as pd
import pyterrier as pt
import string



In [3]:
!rm -rf ./index

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [6]:
input_file = "Answers.json"
query_file = "topics_2.json"
qrel_file = "qrel_1.tsv"

with open(input_file,'r') as file:
    answers = json.load(file)

with open(query_file,'r') as file:
    queries = json.load(file)

In [26]:
#very simple preprocessing, parses inputs with lxml parser as default python parser removed
# way too much of the anwswer text.
def process_input(data):
    for item in data:
        text = item['Text']
        soup = BeautifulSoup(text, "lxml")
        text = soup.get_text()
        item['Text'] = text

In [8]:
process_input(answers)

In [9]:
index_path = "./index"

In [10]:
#define format of incoming data
columns = ['Id','Text','Score']

In [11]:
#reformats answers to be in terrier compatible format
answer_frame = pd.DataFrame(answers, columns=columns)
answer_frame = answer_frame.rename(columns={'Id': 'docno','Text':'text'})

In [ ]:
answer_frame

In [ ]:
pd_indexer = pt.terrier.IterDictIndexer(index_path).index(answer_frame.to_dict(orient='records'))

In [15]:
index = pt.IndexFactory.of(pd_indexer)

In [ ]:
print(index.getCollectionStatistics().toString())

In [18]:
"""def process_queries(queries):
    #this converts the raw json querries into [qid: 'num',query:[token,token,token]]
    processed_queries = pd.DataFrame(columns=['qid', 'query'])

    for index, row in queries.iterrows():
        qid, title = row['Id'], row['Title']
        #THIS IS SOOOO IMPORTANT HOLY CRAP WHY DOES THE "?" CHAR CRASH TERRIER?!?!?!?!?
        title = title.translate(str.maketrans('', '', string.punctuation))
        new_row = {'qid': qid, 'query': title}
        new_row_df = pd.DataFrame([new_row])
        processed_queries = pd.concat([processed_queries, new_row_df], ignore_index=True)
        
    return processed_queries
    """ #deprecated this should work but doesnt...

In [17]:
#processes the queires via the provided class example.
#further expienments should change how we do this
def process_queries(queries):
    lst_q = []
    for item in queries:
        lst_q.append([item['Id'],
        item['Title'].translate(str.maketrans('', '', string.punctuation))])
    processed_queries = pd.DataFrame(lst_q, columns=["qid", "query"])
    return processed_queries

In [18]:
queries = json.load(open(query_file))
processed_queries = process_queries(queries)

In [19]:
#define the retreivers for all the models used

bm_retr = pt.terrier.Retriever(index,controls = {"wmodel": "BM25"},verbose=True,num_results=1000)
dfiz_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFIZ"},verbose=True,num_results=1000)
dfic_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFIC"},verbose=True,num_results=1000)
hiemstralm_retr = pt.terrier.Retriever(index,controls = {"wmodel": "Hiemstra_LM"},verbose=True,num_results=1000)
dfree_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFRee"},verbose=True,num_results=1000)
dfreeKlim_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFReeKLIM"},verbose=True,num_results=1000)
tf_retr = pt.terrier.Retriever(index,controls = {"wmodel": "TF_IDF"},verbose=True,num_results=1000)

In [ ]:
#Testing for two models to make sure things are properly working
bm_res = bm_retr.transform(processed_queries)
dfiz_res = dfiz_retr.transform(processed_queries)

In [20]:
#evaluation
from pyterrier.measures import *
qrels = pt.io.read_qrels(qrel_file)

In [21]:
metrics = ["ndcg", "P_5", "P_10", "P", "map", "bpref","recip_rank"]

In [ ]:
#defines the expirment to run using chosen metrics and excluding queries not in the qrel
res = pt.Experiment(
    [bm_retr,dfiz_retr,dfic_retr,hiemstralm_retr,dfree_retr,dfreeKlim_retr,tf_retr],
    processed_queries,
    qrels,
    eval_metrics=metrics,
    names=["BM25","DFIZ","DFIC","Hiemstra_Lm","DFRee","DFreeKlim","TF-IDF"],
    save_dir="./",
    filter_by_qrels=True,
    test='t',
    baseline=6
)

In [ ]:
print(res)

In [122]:
#formatting results into a dataframe for export
names=["BM25","DFIZ","DFIC","Hiemstra_Lm","DFRee","DFreeKlim","TF-IDF"]
res_df = pd.DataFrame(res)

In [107]:
res_df.to_csv('results.tsv', sep='\t', index=False)

In [ ]:
#runs a per query expirement for top 3 models
per_query = pt.Experiment(
    [bm_retr,tf_retr,dfree_retr],
    processed_queries,
    qrels,
    eval_metrics=["ndcg", "P_5", "bpref","recip_rank"],
    filter_by_qrels=True,
    perquery=True,
    names=["BM25","TF-IDF","DFreeKlim"]
    )

In [130]:
per_query.to_csv('per_q.tsv',sep='\t', index=False)

In [ ]:
#Ad-hoc querry
ad_hoc = "best way to monitor processes"
top_k = 5
top_k_bm = bm_retr.search(ad_hoc).head(top_k)
top_k_df = dfreeKlim_retr.search(ad_hoc).head(top_k)


In [31]:
#for in editor viewing of result information.
df_results = pd.read_csv("results/topic_1_DFreeKlim.res/DFreeKlim.res", sep=' ')
tf_results = pd.read_csv("results/topic_1_TF-IDF.res/TF-IDF.res", sep=' ')
bm_results = pd.read_csv("results/topic_1_BM25.res/BM25.res", sep=' ')